In [ ]:
import tkinter as tk
from tkinter import messagebox
import json
import random

# --- Load questions from JSON file ---
def load_questions(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data['perguntas']

# --- Main app class ---
class ExamSimulatorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Exam Simulator - Night Mode")
        self.root.geometry("700x500")
        self.root.configure(bg='#2E2E2E')  # Dark background
        
        # Config params
        self.num_questions = tk.IntVar(value=5)
        self.penalize_wrong = tk.BooleanVar(value=True)
        self.penalize_unanswered = tk.BooleanVar(value=False)
        self.penalty_type = tk.StringVar(value='0.5')  # Options: '1', '0.5', '0.33'
        
        self.questions = []
        self.current_index = 0
        self.user_answers = {}
        
        self.create_config_ui()
    
    def create_config_ui(self):
        # Clear window
        for widget in self.root.winfo_children():
            widget.destroy()
            
        lbl_title = tk.Label(self.root, text="Configure Your Test", font=("Arial", 20), fg="white", bg='#2E2E2E')
        lbl_title.pack(pady=20)
        
        frame = tk.Frame(self.root, bg='#2E2E2E')
        frame.pack(pady=10)
        
        # Number of questions
        tk.Label(frame, text="Number of Questions:", fg="white", bg='#2E2E2E').grid(row=0, column=0, sticky='w')
        tk.Spinbox(frame, from_=1, to=50, width=5, textvariable=self.num_questions).grid(row=0, column=1, sticky='w')
        
        # Penalize wrong answers
        tk.Checkbutton(frame, text="Penalize Wrong Answers", variable=self.penalize_wrong, fg="white", bg='#2E2E2E', selectcolor='#2E2E2E').grid(row=1, column=0, sticky='w')
        
        # Penalize unanswered questions
        tk.Checkbutton(frame, text="Penalize Unanswered Questions", variable=self.penalize_unanswered, fg="white", bg='#2E2E2E', selectcolor='#2E2E2E').grid(row=2, column=0, sticky='w')
        
        # Penalty type
        tk.Label(frame, text="Penalty Weight:", fg="white", bg='#2E2E2E').grid(row=3, column=0, sticky='w')
        options = ['1', '0.5', '0.33', '0.25']
        penalty_menu = tk.OptionMenu(frame, self.penalty_type, *options)
        penalty_menu.config(bg='#3C3F41', fg='white', highlightthickness=0)
        penalty_menu["menu"].config(bg='#3C3F41', fg='white')
        penalty_menu.grid(row=3, column=1, sticky='w')
        
        # Start button
        btn_start = tk.Button(self.root, text="Start", command=self.start_test, bg='#4B6EAF', fg='white', font=('Arial', 14))
        btn_start.pack(pady=20)
    
    def start_test(self):
        # Load questions
        try:
            all_questions = load_questions('questions/questions.json')
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load questions: {e}")
            return
        
        n = self.num_questions.get()
        if n > len(all_questions):
            messagebox.showwarning("Warning", f"Only {len(all_questions)} questions available, selecting all.")
            n = len(all_questions)
        
        # Randomize and select
        self.questions = random.sample(all_questions, n)
        self.current_index = 0
        self.user_answers = {}
        
        self.show_question()
    
    def show_question(self):
        # Clear window
        for widget in self.root.winfo_children():
            widget.destroy()
        
        if self.current_index >= len(self.questions):
            self.show_results()
            return
        
        q = self.questions[self.current_index]
        
        lbl_num = tk.Label(self.root, text=f"Question {self.current_index+1} of {len(self.questions)}", fg="white", bg='#2E2E2E', font=('Arial', 12))
        lbl_num.pack(anchor='w', padx=20, pady=(10,0))
        
        lbl_text = tk.Label(self.root, text=q['enunciado'], fg="white", bg='#2E2E2E', font=('Arial', 14), wraplength=650, justify='left')
        lbl_text.pack(pady=10, padx=20)
        
        self.answer_var = tk.IntVar(value=-1)
        
        frame_options = tk.Frame(self.root, bg='#2E2E2E')
        frame_options.pack(pady=10, padx=20, anchor='w')
        
        for idx, option in enumerate(q['alternativas']):
            rb = tk.Radiobutton(frame_options, text=option, variable=self.answer_var, value=idx,
                                fg="white", bg='#2E2E2E', selectcolor='#4B6EAF', activebackground='#2E2E2E',
                                font=('Arial', 12), wraplength=600, justify='left')
            rb.pack(anchor='w', pady=2)
        
        frame_nav = tk.Frame(self.root, bg='#2E2E2E')
        frame_nav.pack(pady=20)
        
        if self.current_index > 0:
            btn_prev = tk.Button(frame_nav, text="Previous", command=self.prev_question, bg='#4B6EAF', fg='white')
            btn_prev.pack(side='left', padx=10)
        
        btn_next = tk.Button(frame_nav, text="Next", command=self.next_question, bg='#4B6EAF', fg='white')
        btn_next.pack(side='left', padx=10)
        
        btn_finish = tk.Button(frame_nav, text="End exam", command=self.finish_test, bg='#AF4B4B', fg='white')
        btn_finish.pack(side='left', padx=10)
        
        # Set previous answer if exists
        if self.current_index in self.user_answers:
            self.answer_var.set(self.user_answers[self.current_index])
    
    def prev_question(self):
        self.save_answer()
        if self.current_index > 0:
            self.current_index -= 1
            self.show_question()
    
    def next_question(self):
        self.save_answer()
        if self.current_index < len(self.questions) - 1:
            self.current_index += 1
            self.show_question()
    
    def save_answer(self):
        self.user_answers[self.current_index] = self.answer_var.get()
    
    def finish_test(self):
        self.save_answer()
        self.current_index = len(self.questions)  # to trigger results
        self.show_results()
    
    def show_results(self):
        # Calculate scores
        correct = 0
        wrong = 0
        unanswered = 0
        
        penalty_weight = float(self.penalty_type.get())
        penalize_wrong = self.penalize_wrong.get()
        penalize_unanswered = self.penalize_unanswered.get()
        
        for idx, q in enumerate(self.questions):
            ans = self.user_answers.get(idx, -1)
            if ans == -1:
                unanswered += 1
            elif ans == q['respostaCorreta']:
                correct += 1
            else:
                wrong += 1
        
        penalty = 0
        if penalize_wrong:
            penalty += wrong * penalty_weight
        if penalize_unanswered:
            penalty += unanswered * penalty_weight
        
        score = correct - penalty
        score = max(score, 0)
        
        # Clear window
        for widget in self.root.winfo_children():
            widget.destroy()
        
        lbl_title = tk.Label(self.root, text="===== SCORE =====", fg="white", bg='#2E2E2E', font=('Arial', 20))
        lbl_title.pack(pady=20)
        
        tk.Label(self.root, text=f"Hits: {correct}", fg="white", bg='#2E2E2E', font=('Arial', 14)).pack(pady=5)
        tk.Label(self.root, text=f"Misses: {wrong}", fg="white", bg='#2E2E2E', font=('Arial', 14)).pack(pady=5)
        tk.Label(self.root, text=f"Unanswered: {unanswered}", fg="white", bg='#2E2E2E', font=('Arial', 14)).pack(pady=5)
        tk.Label(self.root, text=f"Final Score: {score:.2f} de {len(self.questions)}", fg="white", bg='#2E2E2E', font=('Arial', 16, 'bold')).pack(pady=15)
        
        frame = tk.Frame(self.root, bg='#2E2E2E')
        frame.pack(pady=20)
        
        btn_restart = tk.Button(frame, text="Try again", command=self.create_config_ui, bg='#4B6EAF', fg='white', font=('Arial', 12))
        btn_restart.pack(side='left', padx=10)

        btn_review = tk.Button(frame, text="Review Test", command=self.review_test, bg='#4BAF6E', fg='white', font=('Arial', 12))
        btn_review.pack(side='left', padx=10)

    def review_test(self):
        # Clear window
        for widget in self.root.winfo_children():
            widget.destroy()
        
        lbl_title = tk.Label(self.root, text="Test Review", fg="white", bg='#2E2E2E', font=('Arial', 20))
        lbl_title.pack(pady=20)
        
        canvas = tk.Canvas(self.root, bg='#2E2E2E', highlightthickness=0)
        scrollbar = tk.Scrollbar(self.root, orient="vertical", command=canvas.yview)
        scroll_frame = tk.Frame(canvas, bg='#2E2E2E')
        
        scroll_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )
        
        canvas.create_window((0, 0), window=scroll_frame, anchor='nw')
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

        for idx, q in enumerate(self.questions):
            user_ans = self.user_answers.get(idx, -1)
            correct_ans = q['respostaCorreta']

            frame_q = tk.Frame(scroll_frame, bg='#2E2E2E', pady=10)
            frame_q.pack(fill='x', padx=20, pady=5, anchor='w')

            tk.Label(frame_q, text=f"Pergunta {idx + 1}: {q['enunciado']}",
                     fg="white", bg='#2E2E2E', wraplength=650, justify='left', font=('Arial', 12, 'bold')).pack(anchor='w')

            for i, alt in enumerate(q['alternativas']):
                color = 'white'
                prefix = ''
                if i == user_ans:
                    if user_ans == correct_ans:
                        color = 'lightgreen'
                        prefix = "✔️ Your correct answer:"
                    else:
                        color = 'tomato'
                        prefix = "❌ Your answer:"
                elif i == correct_ans and user_ans != correct_ans:
                    color = 'lightgreen'
                    prefix = "✅ Correct:"
                else:
                    prefix = " "
                tk.Label(frame_q, text=f"{prefix} {alt}",
                         fg=color, bg='#2E2E2E', wraplength=650, justify='left', font=('Arial', 11)).pack(anchor='w', padx=15)

        tk.Button(self.root, text="Menu", command=self.create_config_ui,
                  bg='#4B6EAF', fg='white', font=('Arial', 12)).pack(pady=20)

# --- Run app ---
if __name__ == '__main__':
    root = tk.Tk()
    app = ExamSimulatorApp(root)
    root.mainloop()
